# PSFs with LUVOIR A telescope

In this notebook, I will demonstrate how to generate a PSF with the LUVOIR A simulator and how to adjust the wavefront (phase) in its pupil plane.

In [ ]:
# Imports
import os
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import hcipy as hc

os.chdir('../../pastis/')
from config import CONFIG_INI
from e2e_simulators.luvoir_imaging import LuvoirAPLC

## Get simple PSF

First we will just instantiate the simulator class and get a simple PSF, both with and without the coronagraph in. We will be working with the "small" apodizer design, to keep things simple, so I would suggest to not change that.

### Instantiate LUVOIR simulator class

In [ ]:
# Parameters
apodizer = 'small'    # coronagraph specification - don't change
sampling = 4          # PSF sampling - 2 would be Nyquist-Shannon

In [ ]:
luvoir = LuvoirAPLC(CONFIG_INI.get('LUVOIR', 'optics_path'), apodizer, sampling)

#### Pupil image

We can display the pupil image easily with `luvoir.aper` and we can plot it. Note how we use a plotting function from `hcipy` instead of `matplotlib`, simply because that's the package we used to develop the simulator.

In [ ]:
plt.figure(figsize=(10, 10))
hc.imshow_field(luvoir.aper)
plt.title('LUVOIR A pupil image')

LUVOIR A has 120 individual hexagonal segments that can each be controlled individually in piston, tip and tilt. The segments are numbered and we can display each segment numer as shown below. This is a slightly hacky way to do it since I didn't write a method for this class to do this yet, but it does the job for now.

In [ ]:
plt.figure(figsize=(10, 10))
hc.imshow_field(luvoir.aper)
plt.title('LUVOIR A segment numbering')
for i, par in enumerate(luvoir.sm.seg_pos):
    pos = par * luvoir.diam
    plt.annotate(s=i+1, xy=pos, xytext=pos, color='white', fontweight='bold')

In order to calculate a perfect PSF *without* the coronagraph in, we need to call `calc_psf()`. The **non-coronagraphic** PSF will be the reference `ref` image that the simulator spits out. The **coronagraphic** image will be called `coro`.

By setting the keyword `display_intermediate=True`, we will get a display of each intermediate optical plane that is of interest to us if we want to have a look at the differen coronagraphic planes. Since we don't really need this though, I will just keep it at its default `False`, so that the PSF compuation happens faster. We do have to set `ref=True` though, otherwise the method only returns a coronagraphic PSF.

In [ ]:
coro, ref = luvoir.calc_psf(ref=True)

#### Reference PSF

Let's display the reference PSF and check it out. Note how I use `norm=LogNorm()` in order to display the image on a log scale.

**NOTE:** 
I will use the terms "*reference PSF*", "*direct PSF*", "*plain PSF*" and "*non-coronagraphic PSF*" interchangeably.

In [ ]:
plt.figure(figsize=(10, 10))
hc.imshow_field(ref, norm=LogNorm())
plt.title('LUVOIR A plain PSF')

What's a bit tricky is that these images are not actually arrays, but they are of the custom type `Field`, which is an `hcipy` thing. It's easy though: a `Field` is just a raveled array. And the `Field` actually has an internal method that lets you cast it into a *shaped* Field, which then behaves exactly like a normal numpy array.

In [ ]:
# Inspect original reference PSF as a hcipy.Field
print('Type of reference PSF: {}'.format(type(ref)))
print('Shape of reference PSF as Field: {}'.format(ref.shape))

# Cast into normal numpy array
ref_array = ref.shaped
print('\nType of reference PSF as array: {}'.format(type(ref_array)))
print('Shape of reference PSF as array: {}'.format(ref_array.shape))

And displaying the shaped Field now works normally with `matplotlib` as we know it:

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(ref_array, norm=LogNorm())
plt.title('Ref PSF as shaped Field = array')

#### Coronagraphic PSF

We can do the same thing with the coronagraphic PSF and display it as an array with `matplotlib`.

In [ ]:
coro_array = coro.shaped
print('Shape of coro PSF as array: {}'.format(coro_array.shape))

plt.figure(figsize=(10, 10))
plt.imshow(coro_array, norm=LogNorm())
plt.title('Coronagraphic PSF, unaberrated and NON-NORMALIZED!!')

The last important thing is that we need to normalize our PSFs to the **peak intensity of the reference PSF**. For this I will define my normalization factor `norm`, which will just be the peak value of the reference PSF. And then anytime I use the coro PSF, I just have to divide it by `norm`. If I now show a colorbar, I can directly read of the contrast in the dark hole.

In [ ]:
# Define normalization factor as max of ref PSF
norm = np.max(ref)

# Display normalized coro PSF
plt.figure(figsize=(10, 10))
plt.imshow(coro_array/norm, norm=LogNorm())
plt.title('Coronagraphic PSF, unaberrated and NORMALIZED!!')
plt.colorbar()

From this point onwards you can just chose whic PSF you want to use, the reference PSF or the *normalized* coronagraphic PSF.

## Introduce wavefront errors

Let's move on to introducing wavefront errors into our pupil, which will change the intensity distribution in the dark hole (DH). This will also allow us to itroduce a defocus diversity.

#### Defocus

First, we need to identify what Zernike to use. Let's take defocus, which has Noll index 4. We can get its Zernike index with the function `hcipy.mode_basis.noll_to_zernike`.

In [ ]:
# Defocus has Noll index 4
noll_index = 4

n_def, m_def = hc.mode_basis.noll_to_zernike(noll_index)
print('Defocus: n={}, m={}'.format(n_def, m_def))

We can now create this Zernike mode as a Field, like above. We will keep it in the raveled format though, so that we can apply it to the pupil of the telescope.

In [ ]:
defocus_mode = hc.mode_basis.zernike(n_def, m_def, D=luvoir.diam, grid=luvoir.aperture.grid)

In [ ]:
# Let's display the Zernike
plt.figure(figsize=(7, 7))
hc.imshow_field(defocus_mode)

This seems to work fine. Now, obviously this is not perfect since this Zernike is made on a circle, while our telescope actually has its funky segmented shape. This should be fine for now though, but just to make sure that they at least have the same size, let's overlay them.

In [ ]:
# Overlap the Zernike with the pupil to make sure array sizes match
pup_zern_test = defocus_mode * luvoir.aperture

plt.figure(figsize=(7, 7))
hc.imshow_field(pup_zern_test)

Looks good enough if you ask me. We now have to actually use the Zernike as a phase input in our propagation. To do this, we can use the method `apply_global_zernike()`. We also have to define **how much** of the Zernike we want to apply to the pupil. This Zernike coefficient `a_4`cis in radians.

In [ ]:
a_4 = 0.5  # rad
luvoir.apply_global_zernike(a_4 * defocus_mode)

Now we can go ahead and calcualte a new pair of ref and coro PSFs. This time though I will set `display_intermediates=True`, so that I can check what is going on in the pupil plane.

In [ ]:
coro_def, ref_def = luvoir.calc_psf(ref=True, display_intermediate=True)

The two plots of interest here are the very first one, which shows the **phase in the pupil** and the very last one, which shows the **intensity distribution** in the final **coronagraphic image**. How about we just put the pupil phase, the ref PSF and the (normalized!) ref PSF next to each other? For this I will have to use another keyword which will actually return all electric fields in all the planes shown above, and I can then pick to display just the **phase** in the **pupil**, next to the two PSFs.

In [ ]:
# Recalculate PSFs with applied Zernike, this time also returning intermediate planes,
# but not displaying them by default
coro_def, ref_def, inter = luvoir.calc_psf(ref=True, return_intermediate='intensity')

I keep forgetting how I pack these intermediate plane returns, so let's check that real quick...

In [ ]:
print(inter.keys())

In [ ]:
type(inter['seg_mirror'])   # this is the phase of the segmented mirror, which is the pupil

Ok got it. Now let's display the pupil phase, the ref image and the coro image next to each other. We will keep using the normalization factor `norm` of the **unaberrated** reference image from further above!

In [ ]:
plt.figure(figsize=(18, 6))

plt.subplot(1, 3, 1)
hc.imshow_field(inter['seg_mirror'])
plt.title('Pupil phase with Zernike')

plt.subplot(1, 3, 2)
hc.imshow_field(ref_def, norm=LogNorm())
plt.title('Reference PSF with Zernike')
plt.colorbar()

plt.subplot(1, 3, 3)
hc.imshow_field(coro_def/norm, norm=LogNorm())
plt.title('Coro PSF with Zernike')
plt.colorbar()   # this is always interesting with a coro PSF

Let's also display the unaberrated ref PSF next to the defocused ref PSF.

In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
hc.imshow_field(ref, norm=LogNorm())
plt.colorbar()

plt.subplot(1, 2, 2)
hc.imshow_field(ref_def, norm=LogNorm())
plt.colorbar()

If the `a_4` is small, the difference will be unnoticable to the eye - the colorbars never lie though! There will always be a higher light level in the DH if there was some defocus introduced.

#### Any Zernike

Let's do this again, but with a different Zernike and a little faster this time. Btw every time you call `apply_global_zernike()`, the pupil phase gets reset to zero, so there is no accidental addition of wavefronts.

In [ ]:
# Pick Noll index
noll_index = 5

n_def, m_def = hc.mode_basis.noll_to_zernike(noll_index)
print('Noll {}: n={}, m={}'.format(noll_index, n_def, m_def))

In [ ]:
# Create the Zernike
zernike_mode = hc.mode_basis.zernike(n_def, m_def, D=luvoir.diam, grid=luvoir.aperture.grid)

hc.imshow_field(zernike_mode)
plt.title('Random Zernike')

In [ ]:
# Decide how much of it you want
a_zern = 0.23   # rad

# Apply to telescope
luvoir.apply_global_zernike(a_zern * zernike_mode)

# Calculate the PSFs and intermediate planes
coro_zern, ref_zern, inter_zern = luvoir.calc_psf(ref=True, return_intermediate='intensity')

In [ ]:
# Display them next to each other
plt.figure(figsize=(18, 6))
plt.suptitle('hcipy display of the Fields')

plt.subplot(1, 3, 1)
hc.imshow_field(inter_zern['seg_mirror'])
plt.title('Pupil phase with Zernike')

plt.subplot(1, 3, 2)
hc.imshow_field(ref_zern, norm=LogNorm())
plt.title('Reference PSF with Zernike')
plt.colorbar()

plt.subplot(1, 3, 3)
hc.imshow_field(coro_zern/norm, norm=LogNorm())
plt.title('Coro PSF with Zernike')
plt.colorbar()   # this is always interesting with a coro PSF

Currently, if the aberration is too big, there can be some funny artifacts show up in the pupil phase image, but I don't know yet if that is a problem or not. And as (I think) I have mentioned before, the segmentation of the pupil seems to get lost somewhere, but if anything, that should make things easier.

Also, remember that you can always use the "shaped" version of one of these "Fields", simply by calling their `.shaped` method. This will probably be needed when working with the images.

In [ ]:
# Display them next to each other - as shaped arrays this time
plt.figure(figsize=(18, 6))
plt.suptitle('Matplotlib display of the shaped arrays')

plt.subplot(1, 3, 1)
plt.imshow(inter_zern['seg_mirror'].shaped)
plt.title('Pupil phase with Zernike')

plt.subplot(1, 3, 2)
plt.imshow(ref_zern.shaped, norm=LogNorm())
plt.title('Reference PSF with Zernike')
plt.colorbar()

plt.subplot(1, 3, 3)
plt.imshow(coro_zern.shaped/norm, norm=LogNorm())
plt.title('Coro PSF with Zernike')
plt.colorbar() 

### Combining errors

In the general case, we will want to be able to apply several Zernikes at once to the pupil phase. This is simply done by adding up all needed Zerenikes into one "master Zernike".

In [ ]:
# Combine defocus and astigmatism from above
master_zern = 0.8 * defocus_mode + 0.3 * zernike_mode

# Then apply this master Zernike to telescope pupil
luvoir.apply_global_zernike(master_zern)

# Calculate the PSFs and intermediate planes
coro_master, ref_master, inter_master = luvoir.calc_psf(ref=True, return_intermediate='intensity')

In [ ]:
# Display them next to each other - as shaped arrays this time
plt.figure(figsize=(18, 6))
plt.suptitle('Matplotlib display of the shaped arrays')

plt.subplot(1, 3, 1)
plt.imshow(inter_master['seg_mirror'].shaped)
plt.title('Pupil phase with Zernike')

plt.subplot(1, 3, 2)
plt.imshow(ref_master.shaped, norm=LogNorm())
plt.title('Reference PSF with Zernike')
plt.colorbar()

plt.subplot(1, 3, 3)
plt.imshow(coro_master.shaped/norm, norm=LogNorm())
plt.title('Coro PSF with Zernike')
plt.colorbar() 